In [119]:
#!pip install translate
!pip install googletrans



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [120]:
import numpy as np

def calculate_score(features):

    #features_PPDB2.0Score: Este puntaje representa la confianza general en la calidad del par de paráfrasis según PPDB 2.0. Un valor más alto indica una mayor probabilidad de que las frases sean paráfrasis válidas.
    #features_AGigaSim: Mide la similitud distribucional entre las frases basándose en contextos observados en el corpus Annotated Gigaword. Valores más altos sugieren una mayor similitud semántica.
    #features_GoogleNgramSim: Calcula la similitud entre las frases según contextos observados en el corpus de Google Ngram. Al igual que AGigaSim, valores más altos indican mayor similitud semántica.
    #features_Equivalence: Predice la probabilidad de que el par de frases represente una equivalencia semántica (es decir, que ambas frases se implican mutuamente). Un valor cercano a 1 sugiere una fuerte equivalencia, mientras que un valor cercano a 0 indica lo contrario.
    #features_Exclusion: Predice la probabilidad de que las frases se excluyan mutuamente en términos semánticos, lo que podría indicar una contradicción. Valores más altos señalan una mayor probabilidad de exclusión.
    #features_ForwardEntailment: Indica la probabilidad de que la primera frase implique a la segunda. Un valor alto sugiere que la primera frase contiene o implica el significado de la segunda.
    #features_Independent: Predice la probabilidad de que las frases sean semánticamente independientes, es decir, que no tengan relación significativa.
    #features_WordCountDiff y features_CharCountDiff: Miden la diferencia en el número de palabras y caracteres entre las frases, respectivamente. Diferencias significativas pueden sugerir variaciones en la complejidad o estructura, aunque no necesariamente en el significado.

    # Asignar pesos a las métricas según su relevancia
    # Calcular la similitud como una combinación ponderada de las métricas
    sum = 0.0
    metrics = 0

    if features.get('features_PPDB2.0Score'):
        min_score = 3
        max_score = 8
        ppdb_score_normalized = (features['features_PPDB2.0Score'] - min_score) / (max_score - min_score)
        #print("sumando: features.get('ppdb_score_normalized') ", ppdb_score_normalized)
        sum += ppdb_score_normalized
        metrics += 1
    if features.get('features_AGigaSim'):
        #print("sumando: features.get('features_AGigaSim') ", features.get('features_AGigaSim'))
        sum += features.get('features_AGigaSim')
        metrics += 1
    if features.get('features_Equivalence'):
        #print("sumando: features.get('features_Equivalence') ", features.get('features_Equivalence'))
        sum += features.get('features_Equivalence')
        metrics += 1
    if features.get('features_Exclusion'):
        #print("sumando: features.get('features_Exclusion') ", 1-features.get('features_Exclusion'))
        sum += (1- features.get('features_Exclusion'))
        metrics += 1
    if features.get('features_Independent'):
        #print("sumando: features.get('features_Independent') ", 1-features.get('features_Independent'))
        sum += (1 - features.get('features_Independent'))
        metrics += 1

    response = sum/metrics
    return response


def parse_features(features_str):
    features = {}
    for item in features_str.split():
        if "=" in item:  # Asegurar que el formato sea clave=valor
            key, value = item.split("=", 1)  # Dividir en clave y valor
            # Manejar valores "NA" convirtiéndolos en None, o convertir a float si es posible
            if value == "NA":
                value = None
            else:
                try:
                    value = float(value)
                except ValueError:
                    pass  # Mantener como string si no se puede convertir
            # Agregar al diccionario con el prefijo 'features_'
            features[f"features_{key}"] = value
    return features


In [121]:

import requests
import json
import time

def get_spanish_traduction(prompt):
    """
    Envía un prompt a Phi para obtener hasta 5 sinónimos de una palabra.

    Args:
        word (str): La palabra para la cual se necesitan sinónimos.

    Returns:
        list: Un array con los sinónimos proporcionados por Phi.
    """

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(
        "http://127.0.0.1:11434/api/generate",
        headers=headers,
        json={"model": "phi3.5", "prompt": prompt}
    )

    if response.status_code == 200:
        # Procesa la respuesta en múltiples líneas
        all_responses = []
        for line in response.text.splitlines():
            # Convierte cada línea a un diccionario y extrae la clave "response"
            try:
                data = json.loads(line)  # Utiliza json.loads en lugar de requests.utils.json
                if 'response' in data:
                    all_responses.append(data['response'])
            except ValueError:
                # En caso de que una línea no sea JSON válido, continuar
                continue

        # Une todos los fragmentos en una sola cadena
        return ''.join(all_responses)
    else:
        # Imprime el código de error si la solicitud falla
        print(f"Error: {response.status_code}, {response.text}")

    return []



def traduccion_simple_phi(word):
    prompt = (
        f"Proporciona la traducción real en español latino para el texto o frase \"{word}\". "
        "Responde solo con un array de un elemento con la respuesta. Sin adicionales."
    )
    print(prompt)
    try:
        response = json.loads(get_spanish_traduction(prompt))
        print(response[0])
        return response[0]
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        time.sleep(2)  # Pausa de 2 segundos

def traduccion_sinonimo_phi(word):
    prompt = (
        f"Proporciona un array con hasta 5 sinónimos más cómunes(en español) de una sola palabra para \"{word}\". "
        "Si encuentras menos de 5 o ningno, incluye solo los disponibles o ninguno. Responde solo con el array."
    )
    return get_spanish_traduction(prompt)

# Ejemplo de uso
#synonyms = traduccion_simple("why are you laughing ?")
#print("Sinónimos:", synonyms)

In [122]:
from googletrans import Translator
import time
import re

def traduccion_simple_local(word):
    try:
        if not word:
            #print("La palabra de entrada es vacía o inválida.")
            return None

        translator = Translator()
        traduccion = translator.translate(word, src='en', dest='es')

        if traduccion.text:
            #print("traduciendo: ", word , " a: ", traduccion.text)
            return traduccion.text
        else:
            #print("No se pudo obtener la traducción.")
            return None

    except Exception as e:
        print(f"Error al traducir: {e}")
        print("Intentando de nuevo sin caracteres especiales: ")

        try:
            translator = Translator()
            translator.raise_exception = True
            traduccion = translator.translate(re.sub(r'[^a-zA-Z0-9]', ' ', word), src='en', dest='es')

            if traduccion.text:
                #print("traduciendo: ", re.sub(r'[^a-zA-Z0-9]', ' ', word) , " a: ", traduccion.text)
                return traduccion.text
            else:
                print("No se pudo obtener la traducción.")
                return word
        except Exception as f:
            print("Segundo intento fallido", {f})
            return word
    #finally:
        # Siempre esperamos un segundo antes de continuar
        #time.sleep(1)


In [123]:
from nltk.corpus import wordnet as wn
import nltk
import re

nltk.download('wordnet')
nltk.download('omw-1.4')

def get_antonyms(word):
    antonyms = set()
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            if lemma.antonyms():
                antonyms.add(lemma.antonyms()[0].name())
    return antonyms

# Procesar una línea del archivo
def process_line(line):
    #print("procesando linea: ", line)
    parts = line.strip().split(" ||| ")
    lhs = parts[0]  # Etiqueta gramatical
    phrase = parts[1]  # Frase original
    paraphrase = parts[2]  # Paráfrasis
    features = parts[3]  # Características
    parsed_features = parse_features(features)
    score = calculate_score(parsed_features)
    alignment = parts[4]  # Alineación
    entailment = parts[5]  # Relación de inferencia (e.g., Equivalence, ForwardEntailment)

    if any(char in (phrase or "") for char in ["[", "]"]) or any(char in (paraphrase or "") for char in ["[", "]"]):
        return None

    # Extraer los antónimos para crear pares contradictorios
    phrase_words = phrase.split()
    contradictory_phrases = []
    for word in phrase_words:
        antonyms = get_antonyms(word)
        for antonym in antonyms:
            contradictory_phrases.append(phrase.replace(word, antonym))

    #print("contra_array: ", contradictory_phrases) #['concentrate', 'centralize', 'centralized']
    
    return {
        "lhs": lhs,
        #"phrase": traduccion_simple_local(phrase),
        #"paraphrase": traduccion_simple_local(paraphrase),
        "phrase": phrase,
        "paraphrase": paraphrase,
        "score": score,
        "contradictions": contradictory_phrases, #['concentrate', 'centralize', 'centralized']
        "alignment": alignment,
        "entailment": entailment,
    }


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Gonzalo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [125]:
from tqdm import tqdm

def process_ppdb_file(file_path):
    num_filas=35
    limit = False
    i = 0
    corpus = {"paraphrases": [], "contradictions": []}
    # Obtener la cantidad total de líneas en el archivo
    
    with open(file_path, "r", encoding="utf-8") as file:
        total_lines = sum(1 for _ in file)
    print("Fin del Conteo")

    with open(file_path, "r", encoding="utf-8") as file:
        with tqdm(total=total_lines, desc="Procesando archivo", unit="línea") as pbar:
            for line in file:
                #Excepciones
                if "www" in line:
                    pbar.update(1)
                    continue

                data = process_line(line)
                if(data == None):
                    pbar.update(1)
                    continue
                
                corpus["paraphrases"].append((data["phrase"], data["paraphrase"],  data["entailment"], data["score"]))
                for contradiction in data["contradictions"]:
#                    corpus["contradictions"].append((data["phrase"], traduccion_simple_local(contradiction), "contradiction", 1 - data["score"]))
                    corpus["contradictions"].append((data["phrase"], contradiction, "contradiction", 1 - data["score"]))

                if i >= num_filas and limit:
                    break
                i = i +1
                pbar.update(1)
    return corpus

ppdb_file = "ppdb-2.0-m-all"
corpus = process_ppdb_file(ppdb_file)


Fin del Conteo


Procesando archivo: 100%|██████████| 9103493/9103493 [11:32<00:00, 13137.84línea/s]


In [128]:
import csv

def export_to_tsv(data, file_name):
    with open(file_name, "w", newline="", encoding="utf-8") as tsvfile:
        writer = csv.writer(tsvfile, delimiter="\t")
        writer.writerow(["Phrase", "Paraphrase/Contradiction", "Type", "Score"])  # Encabezados
        for phrase, related, type, score in data:
            writer.writerow([phrase, related, type, score])

export_to_tsv(corpus["paraphrases"], "paraphrases.tsv")
export_to_tsv(corpus["contradictions"], "contradictions.tsv")